<a href="https://colab.research.google.com/github/minh-chaudang/IntroAI/blob/main/Copy_of_Bloxorz.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [112]:
from copy import copy, deepcopy
import time
import os
import random
import numpy as np

In [113]:
class color:
   PURPLE = '\033[95m'
   CYAN = '\033[96m'
   DARKCYAN = '\033[36m'
   BLUE = '\033[94m'
   GREEN = '\033[92m'
   YELLOW = '\033[93m'
   RED = '\033[91m'
   BOLD = '\033[1m'
   UNDERLINE = '\033[4m'
   END = '\033[0m'

In [114]:
# Create a dictionary to save memory later
move = {None: "Initial Block", 0 : "NONE", -1 : "UP", 1 : "DOWN", -2 : "LEFT", 2 : "RIGHT"}
state = {0 : "STANDING", 1 : "LYING_HORIZONTALLY", 2 : "LYING_VERTICALLY", 3 : "SPLIT"}
square = {0 : "NONE", 1 : "WEAK", 2 : "STRONG", 3 : "HARD_SWITCH_TOGGLE", 4: "HARD_SWITCH_UNOFFABLE", 5 : "SOFT_SWICH_TOGGLE", 6 : "SOFT_SWITCH_UNOFFABLE", 7 : "TELEPORT"}
on = {0 : "OFF", 1 : "ON"}

In [115]:
class Block:
  def __init__(self, parts, last_move = None, parent = None):
    self.parts = sorted(parts, key = lambda x : 1000*x[0] + x[1])
    self.state = self.get_state()
    self.last_move = last_move
    self.parent = parent

  def get_state(self):
    row_diff = abs(self.parts[0][0] - self.parts[1][0])
    col_diff = abs(self.parts[0][1] - self.parts[1][1])

    if row_diff == 0 and col_diff == 0: return 0
    elif row_diff == 0 and col_diff == 1: return 1
    elif row_diff == 1 and col_diff == 0: return 2
    else: return 3
  
  def move(self, instruction, movable_part = None):
    if instruction == 0: return copy.copy(self)

    child_parts = deepcopy(self.parts)
    # Standing
    if self.state == 0:
      if instruction == -1:
        child_parts[0][0] -= 2
        child_parts[1][0] -= 1
      elif instruction == 1:
        child_parts[0][0] += 1
        child_parts[1][0] += 2
      elif instruction == -2:
        child_parts[0][1] -= 2
        child_parts[1][1] -= 1
      else:
        child_parts[0][1] += 1
        child_parts[1][1] += 2
    # Lying horizontally
    elif self.state == 1:
      if instruction == -1:
        child_parts[0][0] -= 1
        child_parts[1][0] -= 1
      elif instruction == 1:
        child_parts[0][0] += 1
        child_parts[1][0] += 1
      elif instruction == -2:
        child_parts[0][1] -= 1
        child_parts[1][1] -= 2
      else:
        child_parts[0][1] += 2
        child_parts[1][1] += 1
    # Lying vertically
    elif self.state == 2:
      if instruction == -1:
        child_parts[0][0] -= 1
        child_parts[1][0] -= 2
      elif instruction == 1:
        child_parts[0][0] += 2
        child_parts[1][0] += 1
      elif instruction == -2:
        child_parts[0][1] -= 1
        child_parts[1][1] -= 1
      else:
        child_parts[0][1] += 1
        child_parts[1][1] += 1
    #Split
    elif self.state == 3: child_parts[movable_part][abs(instruction) - 1] += np.sign(instruction)

    if self.state == 3: return Block(child_parts, (move[instruction], movable_part), self)
    else: return Block(child_parts, move[instruction], self)

  def __eq__(self, other):
    return self.parts == other.parts

In [116]:
class Special_square:
  def __init__ (self, name, position, affects, on = 0):
    self.name = name
    self.position = position
    self.affects = affects
    self.on = on

  def toggle(self):
    self.on = 1 - self.on
  
  def print(self):
    # Weak switch
    if self.name == 3: 
      if self.on == 1: print("Weak switch at", self.position, "is now on, squares", self.affects, "are added!")
      else: print("Weak switch at", self.position, "is now off, squares", self.affects, "are removed!")
    if self.name == 4:
      if self.on == 1: print("Strong switch at", self.position, "is now on, squares", self.affects, "are added!")
      else: print("Strong switch at", self.position, "is now off, squares", self.affects, "are removed!")
    if self.name == 5: print("Teleport at", self.position, "splits blocks into", self.affects[0], "and", self.affects[1])

In [145]:
class Map: 
  # A map stores a 2d-array of squares and its special squares
  def __init__(self, land, specials = None):
    self.land = land
    self.row = len(land)
    self.col = len(land[0])
    self.specials = specials

  def __eq__(self, other):
    return self.land == other.land

  # Find a special square at a postion
  def find_special(self, position):
    return next(x for x in self.specials if x.position == position)

  # Get value of pos on map's length
  def get_value(self, pos):
    return self.land[pos[0]][pos[1]]

  # Check if a block is valid on this map
  def valid_block(self, block):
    # Position not in map, return False
    if not (block.parts[0][0] in range(self.row) and block.parts[1][0] in range(self.row) and block.parts[0][1] in range(self.col) and block.parts[1][1] in range(self.col)): 
      return False
    # One part on 0-square, return False
    if self.get_value(block.parts[0]) <= 0 or self.get_value(block.parts[1]) <= 0: return False
    # Standing on weak square, return False
    if block.state == 0 and self.get_value(block.parts[0]) < 2: return False

    return True

  # Check if a block activates some special squares on this map and return theirs position
  def activated(self, block):
    activated = []
    # Standing on hard ones
    if block.state == 0:
      # Toggle hard switch
      if self.get_value(block.parts[0]) == 3 and block.parent.parts[0] != block.parts[0]: 
        activated.append(deepcopy(block.parts[0]))
      # Unoffable hard switch currently off
      if self.get_value(block.parts[0]) == 4 and block.parent.parts[0] != block.parts[0]:
        unoff_hard = self.find_special(block.parts[0])
        if unoff_hard.on == 0: activated.append(deepcopy(block.parts[0]))
    # One parts on soft ones
    else:
      # Toggle soft switch
      if self.get_value(block.parts[0]) == 5 and block.parent.parts[0] != block.parts[0]: 
        activated.append(deepcopy(block.parts[0]))
      if self.get_value(block.parts[1]) == 5 and block.parent.parts[1] != block.parts[1]: 
        activated.append(deepcopy(block.parts[1]))
      # Unoffable soft switch currently off
      if self.get_value(block.parts[0]) == 6 and block.parent.parts[0] != block.parts[0]:
        unoff_soft = self.find_special(block.parts[0])
        if unoff_soft.on == 0: activated.append(deepcopy(block.parts[0]))
      if self.get_value(block.parts[1]) == 6 and block.parent.parts[0] != block.parts[1]:
        unoff_soft = self.find_special(block.parts[1])
        if unoff_soft.on == 0: activated.append(deepcopy(block.parts[1]))
      if self.get_value(block.parts[0]) == 7 and block.parent.parts[0] != block.parts[0] and block.state != 3:
        activated.append(deepcopy(block.parts[0]))
      if self.get_value(block.parts[1]) == 7 and block.parent.parts[1] != block.parts[1] and block.state != 3:
        activated.append(deepcopy(block.parts[1]))
    return activated

In [146]:
class State:
  def __init__ (self, map, block, parent = None):
    self.map = map
    self.block = block
    self.parent = parent

  def is_goal(self):
    return self.block.state == 0 and self.map.get_value(self.block.parts[0]) == 10

  def move(self, instruction, movable_part = None): 
    child_block = self.block.move(instruction, movable_part)
    if not self.map.valid_block(child_block): return None

    child_map = self.map
    activated = child_map.activated(child_block)

    if len(activated) > 0:
      child_map = deepcopy(self.map)
      for pos in activated:
        special = child_map.find_special(pos)
        # Activate switches
        if special.name in range(3, 7):
          if special.on == 0:
            special.on == 1
            for pos in special.affects:
              child_map.land[pos[0]][pos[1]] = 2
          else:
            special.on == 0
            for pos in special.affects:
              child_map.land[pos[0]][pos[1]] = 0
        else:
          child_block = Block(special.affects, move[instruction], self.block)
    return State(child_map, child_block, self)


  def expand(self):
    children = []
    if self.block.state == 3:
      for i in [-2, -1, 2, 1]:
        child0 = self.move(i, 0)
        if child0 is not None: children.append(child0)
        child1 = self.move(i, 1)
        if child1 is not None: children.append(child1)
    else:
      for i in [-2, -1, 2, 1]:
        child = self.move(i)
        if child is not None: children.append(child)
    return children

  def __eq__(self, other):
    return self.map == other.map and self.block == other.block

In [154]:
class BLOXORZ:
  def __init__ (self, initial_state):
    self.initial_state = initial_state

  def DFShelper(self, stack, visited, loop, max_stack_size):
    while len(stack) > 0:
      max_stack_size = max(len(stack), max_stack_size)
      current = stack.pop()

      # Just to check if initial state is also goal
      if current.is_goal(): return current, loop, max_stack_size

      visited.append(current)
      children = current.expand()
      for child in children:
        if child.is_goal(): return child, loop, max_stack_size
        if child not in visited: 
          stack.append(child)
          # print(child.parent.block.parts, child.block.last_move, child.block.parts)
             
      loop += 1

    return None, loop, max_stack_size

  def DFS(self):
    goal, loop, max_stack_size = self.DFShelper([self.initial_state], [], 0, 0)
    if goal is None: print("No solution")
    else:
      path = [goal]
      while path[-1].parent is not None: path.append(path[-1].parent)
      path.reverse()
      print("DFS executed after", loop, "loops", "with max stack size", max_stack_size)
      for i in range(len(path)): print("Step", i, ":", path[i].block.last_move, path[i].block.parts)

In [160]:
level1 = BLOXORZ(State(Map([[2,2,2,0,0,0,0,0,0,0],
                            [2,2,2,2,2,2,0,0,0,0],
                            [2,2,2,2,2,2,2,2,2,0],
                            [0,2,2,2,2,2,2,2,2,2], 
                            [0,0,0,0,0,2,2,10,2,2], 
                            [0,0,0,0,0,0,2,2,2,0]]), 
                             Block([[1,1],[1,1]])))
level2 = BLOXORZ(State(Map([[0,0,0,0,0,0,2,2,2,2,0,0,2,2,2],
                            [2,2,2,2,0,0,2,2,3,2,0,0,2,10,2],
                            [2,2,5,2,0,0,2,2,2,2,0,0,2,2,2],
                            [2,2,2,2,0,0,2,2,2,2,0,0,2,2,2],
                            [2,2,2,2,0,0,2,2,2,2,0,0,2,2,2],
                            [2,2,2,2,0,0,2,2,2,2,0,0,0,0,0]], 
                            [Special_square(3, [2,2], [[4,4], [4,5]]), Special_square(4, [1,8], [[4,10], [4,11]])]),
                             Block([[3,1],[3,1]])))
level3 = BLOXORZ(State(Map([[0,0,0,0,0,0,2,2,2,2,2,2,2,0,0],
                            [2,2,2,2,0,0,2,2,2,0,0,2,2,0,0],
                            [2,2,2,2,2,2,2,2,2,0,0,2,2,10,2],
                            [2,2,2,2,0,0,0,0,0,0,0,2,2,2,2],
                            [0,0,0,0,0,0,0,0,0,0,0,0,2,2,2]]),
                             Block([[2,1],[2,1]])))
level4 = BLOXORZ(State(Map([[0,0,0,1,1,1,1,1,1,1,0,0,0,0],
                            [0,0,0,1,1,1,1,1,1,1,0,0,0,0],
                            [2,2,2,2,0,0,0,0,0,2,2,2,0,0],
                            [2,2,2,0,0,0,0,0,0,0,2,2,0,0],
                            [2,2,2,0,0,0,0,0,0,0,2,2,0,0],
                            [2,2,2,0,0,2,2,2,2,1,1,1,1,1],
                            [2,2,2,0,0,2,2,2,2,1,1,1,1,1],
                            [0,0,0,0,0,2,10,2,0,0,1,1,2,1],
                            [0,0,0,0,0,2,2,2,0,0,1,1,1,1]]),
                             Block([[5,1],[5,1]])))
level5 = 

In [ ]:
level4.DFS()

In [ ]:
level5 = State([[0,0,0,0,0,0,0,0,0,0,0,2,2,2,2],
                [0,2,2,2,0,0,2,2,5,2,2,2,2,2,2],
                [0,2,2,2,2,0,0,0,0,0,0,0,2,2,2],
                [0,2,2,5,2,0,0,0,0,0,0,0,0,0,0],
                [0,2,2,2,2,0,0,0,0,0,0,0,0,0,0],
                [0,0,0,2,2,2,2,2,2,2,2,2,2,0,0],
                [0,0,0,0,0,0,0,0,0,0,2,2,2,2,2],
                [2,2,2,0,0,0,0,0,0,0,2,2,2,2,2],
                [2,10,2,2,2,2,2,2,2,2,2,2,2,0,0],
                [2,2,2,2,2,0,0,0,0,0,0,0,0,0,0]], Block([[1,12],[1,12]]))

level6 = State([[0,0,0,0,0,2,2,2,2,2,2,0,0,0,0],
                  [0,0,0,0,0,2,0,0,2,2,2,0,0,0,0],
                  [0,0,0,0,0,2,0,0,2,2,2,2,2,0,0],
                  [2,2,2,2,2,2,0,0,0,0,0,2,2,2,2],
                  [0,0,0,0,2,2,2,0,0,0,0,2,2,10,2],
                  [0,0,0,0,2,2,2,0,0,0,0,0,2,2,2],
                  [0,0,0,0,0,0,2,0,0,2,2,0,0,0,0],
                  [0,0,0,0,0,0,2,2,2,2,2,0,0,0,0],
                  [0,0,0,0,0,0,2,2,2,2,2,0,0,0,0],
                  [0,0,0,0,0,0,0,2,2,2,0,0,0,0,0]], Block([[3,0],[3,0]]))

level7 = State([[0,0,0,0,0,0,0,0,0,2,2,2,0,0,0],
                [0,0,0,0,0,0,0,0,0,2,2,2,0,0,0],
                [0,0,0,0,0,0,0,0,0,2,2,2,0,0,0],
                [2,2,2,2,2,2,0,0,0,2,2,2,2,2,2],
                [2,2,2,2,5,2,0,0,0,2,2,2,2,10,2],
                [2,2,2,2,2,2,0,0,0,2,2,2,2,2,2],
                [0,0,0,0,0,0,0,0,0,2,2,2,0,0,0],
                [0,0,0,0,0,0,0,0,0,2,2,2,0,0,0],
                [0,0,0,0,0,0,0,0,0,2,2,2,0,0,0]], Block([[4,1],[4,1]]), [Special_square(5, [4,4], [[1,10], [7,10]])])

In [ ]:
test = BLOXORZ(level7)

# weak = state1.find_special([2,2])

# print(weak.affects)
state1 = State([[0,0,0,0,0,0,0,0,0,2,2,2,0,0,0],
                [0,0,0,0,0,0,0,0,0,2,2,2,0,0,0],
                [0,0,0,0,0,0,0,0,0,2,2,2,0,0,0],
                [2,2,2,2,2,2,0,0,0,2,2,2,2,2,2],
                [2,2,2,2,5,2,0,0,0,2,2,2,2,10,2],
                [2,2,2,2,2,2,0,0,0,2,2,2,2,2,2],
                [0,0,0,0,0,0,0,0,0,2,2,2,0,0,0],
                [0,0,0,0,0,0,0,0,0,2,2,2,0,0,0],
                [0,0,0,0,0,0,0,0,0,2,2,2,0,0,0]], Block([[3,1],[3,1]]), [Special_square(5, [4,4], [[1,10], [7,10]])])
level1 = BLOXORZ(state1)

level1.DFS()

In [85]:

  
# Estimate cost to goal, calculated my average of Manhattan distances of 2 part of the block or Chebychev distance
def cost_to_goal(self, current):
  #Average Manhattan
  average = 1/2 * (abs(current.parts[0][0] - self.goal[0]) + abs(current.parts[1][0] - self.goal[0]) + abs(current.parts[1][0] - self.goal[1]) + abs(current.parts[1][1] - self.goal[1]))
  return 1/average
  # Chebychev
  # max1 = max(abs(current.parts[0][0] - self.goal[0]), abs(current.parts[0][1] - self.goal[1]))
  # max2 = max(abs(current.parts[1][0] - self.goal[0]), abs(current.parts[1][1] - self.goal[1]))
  # return 1/max(max1, max2)

def move_process(self, process):
  current = copy.copy(self.initial_block)
  for move in process: 
    current = current.move(move)
    if not self.is_valid(current): 
      return -1
  return current


In [ ]:
class Genetic():
  population = []
  prob = []
  def __init__(self, game, chromosome_length, population_capacity):
    self.game = game
    self.chromosome_length = chromosome_length
    self.population_capacity = population_capacity
    self.initPopulation()
    self.probCal()

  def initIndividual(self):
    chromosome = []
    for i in range(self.chromosome_length):
      # if chromosome is empty or having NONE as the last gen, add an arbitrary
      if len(chromosome) == 0 or chromosome[-1] == 0: chromosome.append(random.choice((0,1,2,3,4)))
      # if the last gen is UP, avoid DOWN
      elif chromosome[-1] == 1: chromosome.append(random.choice((0,1,3,4)))
      # if the last gen is DOWN, avoid UP
      elif chromosome[-1] == 2: chromosome.append(random.choice((0,2,3,4)))
      # if the last gen is LEFT, avoid RIGHT
      elif chromosome[-1] == 3: chromosome.append(random.choice((0,1,2,3)))
      # if the last gen is LEFT, avoid RIGHT
      else: chromosome.append(random.choice((0,1,2,4)))
    return chromosome
  
  def initPopulation(self):
    while len(self.population) < self.population_capacity:
      individual = self.initIndividual()
      final = self.game.move_process(individual)
      if final != -1: self.population.append(individual)

  def probCal(self):
    for individual in self.population:
      final = self.game.move_process(individual)
      self.prob.append(self.game.cost_to_goal(final))
    _sum = sum(self.prob)
    for i in range(len(self.prob)): self.prob[i] = self.prob[i] / _sum;

  #Seclect a parent in population
  def select(self):
    rannum = random.uniform(0, 1)
    for i in range (self.population_capacity):
      rannum = rannum - self.prob[i]
      if rannum < 0: return self.population[i-1]

  def cross_over(self):
    parent1 = self.select()
    parent2 = self.select()

    ran_index = random.ranint(0, self.chromosome_length)

    for i in range(ran_index, self.chromosome_length): parent1[i] = parent2[i]

    return parent1

  def mutate(self, chromosome):
     ran_index = random.ranint(0, self.chromosome_length)
     chromosome[ran_index] = random.choice((0,1,2,3,4))

  def populationUpdate(self):
    next_population = []

    # Cross over
    while (len(next_population) < self.population_capacity):
      child = self.cross_over()
      if self.game.move_process(child) != -1: next_population.append(child)
    # It is better no mutation
    
    for child in next_population: self.mutate(child)

    self.population = next_population
    self.probCal

  # Loop for 100 times
  def go(self):
    for i in range(100):
      for individual in self.population:
        if self.game.move_process(individual).is_goal(): return individual
      self.populationUpdate()

    return "Cannot find Solution"